# Overdispersed Poisson Bootstrap Model in Loss Reserving

This notebook is based on CAS Monograph series Number 4 "Using the ODP Bootstrap Model : A Practitioner's Guide" by Mark R. Shapland. This notebook is made because I have an itch to do the coding (or in other words, I can't hacked this problem in Microsoft Excel). Data that being used is the Worker Compensation Data from CAS website.

# Notation 

1. $i$ : accident year with $i = 0,1,2,\dots, I$
2. $j$ : development year with $j = 0,1,2,\dots, I$
3. $Y_{i,j}$ : incremental claim from accident year $i$ in development year $j$
4. $C_{i,j}$ : cumulative claim from accident year $i$ in development year $j$

# Mathematical Background

We assume that incremental claim, $Y_{i,j}$ for all $i+j \leq I$ comes from a probability distribution function. And let
\begin{align}
\label{first equation}
E[Y_{i,j}] &= \mu_{i,j} \\
\text{Var}\left(Y_{i,j}\right) &= \phi E[X_{i,j}] = \phi \mu_{i,j}^z
\end{align}
and 
\begin{equation}
\log{\left(\mu_{i,j}\right)} = \eta_{i,j}
\end{equation}
with $\eta_{i,j} = c + \alpha_i + \beta_j$, for all $i = 0,1,\dots I$ and $j = 0,1, \dots I$.
We also assume that $\alpha_0 = \beta_0 = 0$ (why ?)

The power, $z$, is used to specify the error distribution with:

| Power | Distribution |
|---|---|
| $z =0$ | Normal |
| $z =1$ | Poisson |
| $z =2$ | Gamma |
| $z =3$ | Inverse Gaussian |

For simplicity, we assume that $c = 0$ and for this notebook purposes, we choose $z=1$.

## Parameter Estimation

We estimate all of the parameters using Iteratively Weighted Least Square, to solve the parameters. Note from equation (\ref{first equation}) above we would have a system of linear equation as follow:  
\begin{align*}
\log{\left(Y_{0,0}\right)} &= \alpha_0 \\
\log{\left(Y_{0,1}\right)} &= \alpha_0 + \beta_1\\
\log{\left(Y_{0,2}\right)} &= \alpha_0 + \beta_1 + \beta_2 \\
&\vdots \\
\log{\left(Y_{0,I}\right)} &= \alpha_0 + \beta_1 + \beta_2 + \dots + \beta_I \\
\log{\left(Y_{1,0}\right)} &= \alpha_1 \\ 
\log{\left(Y_{1,1}\right)} &= \alpha_1 + \beta_1\\
\log{\left(Y_{1,2}\right)} &= \alpha_1 + \beta_1 + \beta_2 \\
&\vdots \\
\log{\left(Y_{0,I-1}\right)} &= \alpha_1 + \beta_1 + \beta_2 + \dots + \beta_{I-1} \\ 
\log{\left(Y_{I,1}\right)} &= \alpha_{I}
\end{align*}
And we could write it in matrix form as below:
\begin{align}
\mathbf{y} = \begin{bmatrix} \log{\left(Y_{0,0}\right)} \\ \log{\left(Y_{0,1}\right)}\\ \log{\left(Y_{0,2}\right)} \\ \vdots \\ \log{\left(Y_{0,I}\right)}
\log{\left(Y_{1,0}\right)} \\
\log{\left(Y_{1,1}\right)} \\
\log{\left(Y_{1,2}\right)} \\ 
\vdots \\
\log{\left(Y_{0,I-1}\right)} \\ 
\log{\left(Y_{I,1}\right)}
\end{bmatrix}, \mathbf{x} = \begin{bmatrix} \alpha_1 \\ \alpha_2 \\ \vdots \\ \alpha_{10} \\ \beta_2 \\ \beta_3 \\ \vdots \\ \beta_{10} \end{bmatrix}
\end{bmatrix}
\end{align}
and $\mathbf{A}$ is the corresponding matrix with values 0 or 1.
\begin{align}
\mathbf{y} = \mathbf{A}\mathbf{x}
\end{align}
with $\mathbf{Y}$ is a column vector of incurred claim, $\mathbf{A}$ is the matrix design (more like indicator matrix) and $\mathbf{x}$ is column vector of the parameters.
We are going to use iteratively weighted least squares to solve the for the parameters in the $\mathbf{x}$.

After we have the parameters, then we are going to fit the model on to the data and we are going to calculate the unscaled Pearson residuals, $r_{i,j}$, as well as the scale (dispersion), $\phi$ parameter with equation below
\begin{equation}
r_{i,j} := \frac{Y_{i,j} - \mu_{i,j}}{\sqrt{\mu_{i,j}^z}}
\end{equation}
and 
\begin{equation}
\phi = \frac{\sum_{(i,j) \in \{i+j\leq I\}}{r_{i,j}^2}}{N-p}
\end{equation}
with $N$ is number of observations and $p$ is number of parameters.

## Packages

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.precision", 4)

# Importing the Data

In [61]:
data = pd.read_csv('data\wkcomp_pos.csv')
data.head()

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
0,86,Allstate Ins Co Grp,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872
1,86,Allstate Ins Co Grp,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872
2,86,Allstate Ins Co Grp,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872
3,86,Allstate Ins Co Grp,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872
4,86,Allstate Ins Co Grp,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872


In [62]:
data.describe()

,GRCODE,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
count,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,1.3200e+04
mean,17153.0530,1992.5000,1997.0000,5.5000,11532.0483,8215.7373,1570.1256,18438.4697,1812.3386,16626.1288,0.7273,3.9714e+04
std,12512.2061,2.8724,4.0622,2.8724,35595.5602,25714.0815,7259.0221,51830.7031,6666.6631,48941.7241,0.4454,1.3013e+05
min,86.0000,1988.0000,1988.0000,1.0000,-59.0000,-338.0000,-4621.0000,-6518.0000,-3522.0000,-9731.0000,0.0000,0.0000e+00
25%,8526.0000,1990.0000,1994.0000,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.1100e+02
50%,14110.0000,1992.5000,1997.0000,5.5000,544.0000,351.5000,5.0000,1419.0000,144.5000,827.0000,1.0000,2.7320e+03
75%,26983.2500,1995.0000,2000.0000,8.0000,6526.5000,4565.0000,259.2500,11354.2500,1141.0000,9180.5000,1.0000,1.9266e+04
max,44300.0000,1997.0000,2006.0000,10.0000,367404.0000,325322.0000,145296.0000,421223.0000,78730.0000,418755.0000,1.0000,1.0901e+06


In [63]:
data.dtypes

GRCODE                int64
GRNAME               object
AccidentYear          int64
DevelopmentYear       int64
DevelopmentLag        int64
IncurLoss_D           int64
CumPaidLoss_D         int64
BulkLoss_D            int64
EarnedPremDIR_D       int64
EarnedPremCeded_D     int64
EarnedPremNet_D       int64
Single                int64
PostedReserve97_D     int64
dtype: object

In [64]:
print(f'Number of Company {len(data.GRCODE.unique())}')

Number of Company 132


Let just focus on 1 Company with GRCODE 337

In [65]:
df = data[data['GRCODE'] == 337]
df

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
100,337,California Cas Grp,1988,1988,1,62679,9558,24619,104437,4658,99779,0,209415
101,337,California Cas Grp,1988,1989,2,64000,22778,13092,104437,4658,99779,0,209415
102,337,California Cas Grp,1988,1990,3,65607,33298,13020,104437,4658,99779,0,209415
103,337,California Cas Grp,1988,1991,4,65292,40348,11540,104437,4658,99779,0,209415
104,337,California Cas Grp,1988,1992,5,62250,45146,9513,104437,4658,99779,0,209415
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,337,California Cas Grp,1997,2002,6,59809,48438,2786,48052,2119,45933,0,209415
196,337,California Cas Grp,1997,2003,7,59230,50775,1358,48052,2119,45933,0,209415
197,337,California Cas Grp,1997,2004,8,59232,52694,1017,48052,2119,45933,0,209415
198,337,California Cas Grp,1997,2005,9,59300,54217,781,48052,2119,45933,0,209415


We are going to show the data in run-off triangle and note, we assume we are in the year of 1997. Meaning that we assume for every data that being collected beyond 1997 is unknown.

In [66]:
df = df[df['DevelopmentYear'] <= 1997]

In [67]:
tr_inc = pd.pivot_table(df, values='IncurLoss_D', index='AccidentYear',
                       columns=['DevelopmentLag'], aggfunc=np.sum)
tr_inc

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,62679.0,64000.0,65607.0,65292.0,62250.0,61185.0,59387.0,57138.0,53473.0,53261.0
1989,62058.0,56660.0,56598.0,56831.0,55294.0,53481.0,51379.0,48446.0,48300.0,NaN
1990,63955.0,61523.0,63166.0,61402.0,59474.0,57613.0,57161.0,56971.0,NaN,NaN
1991,73918.0,75094.0,73188.0,71199.0,71056.0,70831.0,70532.0,NaN,NaN,NaN
1992,77143.0,75133.0,73284.0,73155.0,68284.0,67845.0,NaN,NaN,NaN,NaN
1993,79925.0,78263.0,78163.0,68546.0,67697.0,NaN,NaN,NaN,NaN,NaN
1994,73628.0,73554.0,70736.0,69720.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,69521.0,80281.0,79381.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,69720.0,73181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
tr_cum = tr_inc.cumsum(axis = 1)
tr_cum

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,62679.0,126679.0,192286.0,257578.0,319828.0,381013.0,440400.0,497538.0,551011.0,604272.0
1989,62058.0,118718.0,175316.0,232147.0,287441.0,340922.0,392301.0,440747.0,489047.0,NaN
1990,63955.0,125478.0,188644.0,250046.0,309520.0,367133.0,424294.0,481265.0,NaN,NaN
1991,73918.0,149012.0,222200.0,293399.0,364455.0,435286.0,505818.0,NaN,NaN,NaN
1992,77143.0,152276.0,225560.0,298715.0,366999.0,434844.0,NaN,NaN,NaN,NaN
1993,79925.0,158188.0,236351.0,304897.0,372594.0,NaN,NaN,NaN,NaN,NaN
1994,73628.0,147182.0,217918.0,287638.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,69521.0,149802.0,229183.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,69720.0,142901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Basic Chain Ladder

In [69]:
tr_cum.iloc[0,:9]

DevelopmentLag
1     62679.0
2    126679.0
3    192286.0
4    257578.0
5    319828.0
6    381013.0
7    440400.0
8    497538.0
9    551011.0
Name: 1988, dtype: float64

In [70]:
tr_cum.shape

(10, 10)

In [71]:
def dev_fac(triangle = tr_cum, i = 0, shape = tr_cum.shape[1]):
    return np.sum(tr_cum.iloc[:shape - i - 1, i+1])/np.sum(tr_cum.iloc[:shape - i -1, i])

In [72]:
dev_factor = pd.DataFrame(data = [dev_fac(i = k) for k in range(9)], index = np.arange(0,9), columns = ['fk'])
dev_factor

,fk
0,2.0081
1,1.4969
2,1.3197
3,1.2346
4,1.1887
5,1.1564
6,1.1293
7,1.1085
8,1.0967


In [73]:
ult_factor = dev_factor.sort_index(ascending=False).cumprod()
ult_factor

,fk
8,1.0967
7,1.2156
6,1.3728
5,1.5876
4,1.8871
3,2.3299
2,3.0746
1,4.6023
0,9.2419


In [94]:
print(f'Total Reserve from Chain Ladder Method {np.array(tr_cum.max(axis = 1)[1:].array).dot(ult_factor-1)[0]}')

Total Reserve from Chain Ladder Method 2711890.673831593


# Overdispersed Poisson Model

Let $$\mathbf{y} = \begin{bmatrix} \log{Y_{1,1}} \\ \log{Y_{1,2}} \\ \vdots \\ \log{Y_{1,10}} \\ \log{Y_{2,1}} \\ \vdots \\ \log{Y_{2,9}} \\ \log{Y_{3,1}} \\ \vdots\\ \log{Y_{10,10}}\end{bmatrix}$$ and $$\mathbf{x} = \begin{bmatrix} \alpha_1 \\ \alpha_2 \\ \vdots \\ \alpha_{10} \\ \beta_2 \\ \beta_3 \\ \vdots \\ \beta_{10} \end{bmatrix}$$. 
Note that $\mathbf{y} \in \mathbb{R}^{55}$ and $\mathbf{\beta} \in \mathbb{R}^{19}$

In [76]:
df_odp = df.copy()
df_odp = df_odp[['AccidentYear', 'DevelopmentLag', 'IncurLoss_D']]
df_odp['LogIncurLoss_D'] = np.log(df_odp['IncurLoss_D'])
y = df_odp['LogIncurLoss_D'].values
X = np.zeros((55,19))

In [77]:
i = 0
j = 9
temp = df_odp['AccidentYear'].iloc[0]
for n in range(len(df_odp)):
    if temp != df_odp['AccidentYear'].iloc[n] :
        i += 1
        temp = df_odp['AccidentYear'].iloc[n]
    X[n,i] = 1
    if df_odp['DevelopmentLag'].iloc[n] == 1 :
        j = 9
    else :
        j += 1
        X[n, j] = 1

In [78]:
np.eye(55)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [79]:
def irls(y = y, X = X, iter = 1000):
    weight = np.eye(55)
    params = np.array([1]*X.shape[1])
    for n in range(iter):    
        new_params = np.linalg.inv(X.T.dot(weight).dot(X)).dot(X.T).dot(weight).dot(y)
        for k in range(weight.shape[0]):
            weight[k,k] = 1/(max(1e-4,np.abs(y[k] - X[k,:].dot(new_params))))
        params = new_params
    return params

In [80]:
odp_params = irls()
odp_params

array([ 1.11043388e+01,  1.09756284e+01,  1.10659350e+01,  1.12234605e+01,
        1.12238781e+01,  1.12647948e+01,  1.11940751e+01,  1.12902528e+01,
        1.11749491e+01,  1.08231924e+01,  3.03541886e-03, -1.32138607e-02,
       -4.07373031e-02, -7.23461137e-02, -8.85466974e-02, -1.12407972e-01,
       -1.51114149e-01, -2.03924139e-01, -2.21379188e-01])

In [81]:
df_odp['ODP_LogIncur'] = X.dot(odp_params)
df_odp['ODP_Incur'] = np.exp(df_odp['ODP_LogIncur'])
df_odp['pearson_resid'] = (df_odp['IncurLoss_D'] - df_odp['ODP_Incur'])/np.sqrt(df_odp['ODP_Incur'])

In [82]:
df_odp.head()

,AccidentYear,DevelopmentLag,IncurLoss_D,LogIncurLoss_D,ODP_LogIncur,ODP_Incur,pearson_resid
100,1988,1,62679,11.0458,11.1043,66458.8888,-14.6623
101,1988,2,64000,11.0666,11.1074,66660.9259,-10.3062
102,1988,3,65607,11.0914,11.0911,65586.4869,0.0801
103,1988,4,65292,11.0866,11.0636,63805.9369,5.8831
104,1988,5,62250,11.0389,11.0320,61820.6487,1.7268


In [83]:
phi = np.sum(df_odp['pearson_resid']**2)/(df_odp.shape[0] - len(odp_params))
print(f'Phi value is {phi}')

Phi value is 107.37322423339778


In [84]:
adj_factor = np.sqrt(df_odp.shape[0]/(df_odp.shape[0] - len(odp_params)))
print(f'Degree of Freedom adjustment factor is {adj_factor}')                    

Degree of Freedom adjustment factor is 1.2360330811826103


Now we would like to calculate the reserve and the ultimate claim from ODP Model

In [85]:
reserve = data[(data['GRCODE'] == 337) & (data['DevelopmentYear'] > 1997)]
reserve.head()

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
119,337,California Cas Grp,1989,1998,10,48162,46483,411,88883,3773,85110,0,209415
128,337,California Cas Grp,1990,1998,9,56634,54440,821,85956,3769,82187,0,209415
129,337,California Cas Grp,1990,1999,10,56368,54857,379,85956,3769,82187,0,209415
137,337,California Cas Grp,1991,1998,8,70260,67783,821,99339,4342,94997,0,209415
138,337,California Cas Grp,1991,1999,9,70104,68323,434,99339,4342,94997,0,209415


In [86]:
reserve.shape

(45, 13)

In [87]:
X_

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.

In [88]:
X_ = np.zeros((45,19))
i = 1
j = 9
temp = reserve.AccidentYear.iloc[0]
for n in range(X_.shape[0]):
    if temp != reserve.AccidentYear.iloc[n]:
        i += 1
        temp = reserve.AccidentYear.iloc[n]
    X_[n, i] = 1
    X_[n, 10:9+reserve.DevelopmentLag.iloc[n]] = 1 

In [91]:
reserve = reserve[['AccidentYear', 'DevelopmentLag', 'IncurLoss_D']]
reserve['ODP_Incur'] = np.exp(X_.dot(odp_params))
print(f'Total Reserve from ODP Model {reserve["ODP_Incur"].sum()}')

Total Reserve from ODP Model 2051167.5757406428
